# 第一題 #
讀取2017所有表特版的文章

In [1]:
import requests
import datetime
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import codecs
import time
END_POST = '[正妹] 新年 第889彈'

# function part #

* check_date_from_page
傳進頁面的soup，檢查是否存在12/31號的文章
若有，回傳該文章之soup

* get_website
傳入url，回傳該網站的網站位置

In [2]:
def check_date_from_page(cur_article):
    """

    """
    def check_date(soup):
        """
        input beautifuy object, check date
        must find last
        """
        # get date
        date_string = soup.find('div', 'date').string.strip()
        date = datetime.datetime.strptime(date_string, "%m/%d")
        if date.month != 12 or date.day != 31:
            return False
        else:
            return True

    cur_article = cur_article.find('div', 'r-ent')
    while cur_article is not None:
        if not check_date(cur_article):
            # next
            cur_article = cur_article.find_next('div', 'r-ent')
        else:
            # find last match
            while cur_article is not None:
                prev_article = cur_article
                cur_article = cur_article.find_next('div', 'r-ent')
                # got 1/1
                if not check_date(cur_article):
                    return prev_article
            return prev_article
    return False


def get_website(url):
    """
    get the website from url
    """
    site = urlparse(url)
    site_netloc = site.scheme + '://' + site.netloc
    return site_netloc


def get_response(url):
    response = requests.get(url)
    while not response.ok:
        response = requests.get(url)
    return response


def get_prev_page_url(root_soup):
    return website + root_soup.find('div', id='action-bar-container').find('div', 'btn-group-paging').find_all('a')[1]['href']

* 沒網址的不用爬，可忽略
* 爬2017年的所有文章
* 忽略分類為公告的文章
* output:
* 1. 所有文章 
  2. 所有爆文
* 檔案內容 - 日期,標題,url（無空格

## 結束條件 ##
https://www.ptt.cc/bbs/Beauty/M.1483208018.A.411.html

# main function start #

In [3]:
t0 = time.time()
# find the 12/31 article
url = "https://www.ptt.cc/bbs/Beauty/index.html"
# get ptt's website
website = get_website(url)
# first flag
first_flag = True
############### loading ####################
response = get_response(url)
root_soup = BeautifulSoup(response.text, 'lxml')
############### end loading ################
find_soup = check_date_from_page(root_soup)

while not find_soup:
    # find next page
    print('page: ' + url + ' not find!')
    print('get next page...')
    # prev page's url
    url = get_prev_page_url(root_soup)

    # if this is prist page, than direct minus 300 ...
    if first_flag == True:
        print("first page, direct jump ...")
        # got the number part
        match = re.search(r'(\d+)(?:.html$)', url)
        number = match[1]
        number = int(number) - 300
        # replace back
        url = re.sub(r"(\d+)(.html$)", str(number) + r'\2', url)
        print('jump to: ', url)
        first_flag = False

    # loading
    response = get_response(url)
    root_soup = BeautifulSoup(response.text, 'lxml')
    find_soup = check_date_from_page(root_soup)

print('find!')

# record time
t1 = time.time()
time_file = codecs.open("exec_time.txt", "a", "utf-8")
time_file.write(
    'total time for get the proper 2017 last post: ' + str(t1 - t0) + '\n')
time_file.close()

page: https://www.ptt.cc/bbs/Beauty/index.html not find!
get next page...
first page, direct jump ...
jump to:  https://www.ptt.cc/bbs/Beauty/index2352.html
page: https://www.ptt.cc/bbs/Beauty/index2352.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2351.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2350.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2349.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2348.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2347.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2346.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2345.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2344.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2343.html not find!
get next page...
page: https://www.ptt.cc/bbs/Beauty/index2342.html not find!
ge

# function for next part of code #


In [4]:
def get_date(soup):
    date_string = soup.find('div', 'date').string.strip()
    return date_string


def get_title(cur_soup):
    if cur_soup.find('div', 'title').find('a'):
        title = ''
        for string in cur_soup.find('div', 'title').a.strings:
            # if @ occur, need specical parse
            if string == '[email protected]':
                title = title + '@' + string['data-cfemail']
            else:
                title = title + string
        return title
    else:
        return False


def write_info(all_file, pop_file, soup):
    """soup is on <div r-ent>"""
    def check_title_string(title):
        """title is a string, true mean this is what we need"""
        match = re.search(r'\[公告\]', title)
        if not match:
            return True
        else:
            return False

    # check if title.a exist
    if not soup.find('div', 'title').find('a'):
        return False
    # check title contain [公告] or not
    title = get_title(soup)
    if check_title_string(title):
        # grab date, topic, url
        date_temp = soup.find('div', 'date').string.strip()
        # remove /
        date = re.sub(r'[/]', '', date_temp)
        # title
        url = website + soup.find('div', 'title').find('a')['href']
        # merge to one line
        write_line = ','.join([date, title, url])
        # if 爆, write to pop_file
        thumb_number = soup.find('div', 'nrec')
        if thumb_number.find('span'):
            thumb_number = thumb_number.span.string.strip()
        else:
            tuhmb_number = ''
        if thumb_number == '爆':
            pop_file.write(write_line + '\n')
        all_file.write(write_line + '\n')
        # write to all_file

In [5]:
t2 = time.time()
# start soup
# root_soup is current soup on root
start_soup = find_soup
# open file
all_file = codecs.open("all_articles.txt", "w", "utf-8")
pop_file = codecs.open("all_popular.txt", "w", "utf-8")

# first page is special case
write_info(all_file, pop_file, start_soup)
for cur_soup in start_soup.find_all_previous('div', 'r-ent'):
    if get_title(cur_soup):
        print('write file for "' + get_title(cur_soup) + '"')
    write_info(all_file, pop_file, cur_soup)

# second page start
is_continued = True
while is_continued:
    # to next page
    url = get_prev_page_url(root_soup)
    response = get_response(url)
    root_soup = BeautifulSoup(response.text, 'lxml')

    # loop root_soup
    for cur_soup in reversed(root_soup.find_all('div', 'r-ent')):
        title = get_title(cur_soup)
        if title:
            date = get_date(cur_soup)
            print('write file for "' + date + " " + title + '"')
        write_info(all_file, pop_file, cur_soup)
        # ending point

        if title == END_POST:
            is_continued = False
            break
    is_continued == False
all_file.close()
pop_file.close()

# record time
t3 = time.time()
time_file = codecs.open("exec_time.txt", "a", "utf-8")
time_file.write('total time for crawl all 2017 post: ' +
                str(t3 - t2) + '\n')
time_file.write('total exec time: ' + str(t3 - t0) + '\n')

time_file.close()

write file for "[正妹] 2018 世界新車大展 Day 1"
write file for "[正妹] 2018"
write file for "[正妹] 奶特"
write file for "[正妹] 韓流"
write file for "[正妹] 彩彩小萌獸"
write file for "[正妹] 柏木由紀"
write file for "[正妹] 俄羅斯暗黑女模Helga Lovekaty"
write file for "[正妹] 彩美旬果"
write file for "[正妹] 秀氣長髮 "
write file for "[神人] 台北車展Polo beats DJ"
write file for "Re: [神人] hotels.com廣告女主角"
write file for "[正妹] 台北車展lexus正妹"
write file for "[正妹] 韓流女團藝人"
write file for "12/31 [神人] 自問自答"
write file for "12/31 [神人]台北車展land rover的sg"
write file for "12/31 [正妹] 跨年前發一波"
write file for "12/31 [帥哥] 最近好像流行這種"
write file for "12/31 [正妹] 企業排球聯賽 中纖美女主攻手"
write file for "12/30 [正妹] 無題"
write file for "12/30 [正妹] 世界第衣美"
write file for "12/30 [正妹] 高畑充希 Takahata Mitsuki"
write file for "12/30 [正妹] 兩個高三生"
write file for "12/30 [神人] 柯批剪綵旁的正妹"
write file for "12/30 [正妹] Madison Iseman"
write file for "12/30 [正妹] 李元玲"
write file for "12/30 [帥哥] 正太控請進"
write file for "12/30 [正妹] 浜辺美波"
write file for "12/30 [正妹] IU"
write file for "12/30 [正妹] 阿旦"
w

write file for "12/20 Re: [正妹] 筧美和子"
write file for "12/20 [正妹] 覺得不錯"
write file for "12/20 [神人] 福斯show girl"
write file for "12/20 [正妹] 筧美和子"
write file for "12/20 [神人] 一張圖求神"
write file for "12/20 [正妹] 神七"
write file for "12/20 [正妹] 馬友蓉?????"
write file for "12/20 [正妹] 車模"
write file for "12/20 [神人] 跪求神這位妹子"
write file for "12/20 [正妹] 小倉優香"
write file for "12/20 [正妹] 睡美人"
write file for "12/20 [正妹] 唉呦……不錯哦"
write file for "12/20 Re: [正妹] AKB48 島崎遙香"
write file for "12/20 [正妹] 還可以"
write file for "12/19 [正妹] 好身材"
write file for "12/19 [正妹] Linda"
write file for "12/19 [正妹] 就 阿純"
write file for "12/19 [正妹] 痴的氣息"
write file for "12/19 [神人] 資訊展美女sg"
write file for "12/19 [正妹] 竹東高中"
write file for "12/19 [正妹] 覺得優"
write file for "12/19 [神人] 笑容"
write file for "12/19 [正妹] 銘傳大學"
write file for "12/19 [帥哥] NBA最帥沒有之一:Kobe Bryant"
write file for "12/19 [正妹] 黃芷嫣"
write file for "12/19 [神人] 想請問這位長腿妹"
write file for "12/19 [正妹] 麻亜里"
write file for "12/19 [正妹] 有氣質的外國人"
write file for "12/19 Re: [帥

write file for "12/07 [正妹] 甜美氣質美女"
write file for "12/07 [正妹] 海軍之花~黃靜昀"
write file for "12/07 Re: [正妹] 祈錦鈅 就一張"
write file for "12/07 [正妹] 輕熟姊姊"
write file for "12/07 [正妹] 布莉安娜海德布蘭德"
write file for "12/07 [正妹] 祈錦鈅 就一張"
write file for "12/07 [正妹] 職業籃球啦啦隊"
write file for "12/07 [正妹] Pool 泳池"
write file for "12/07 [神人] 神一位資訊月展場 NEC 的 SG"
write file for "12/07 [正妹] 是記者耶…………"
write file for "12/07 [正妹] 喜歡短髮……"
write file for "12/07 [正妹] 別把螢幕用的白白的…………"
write file for "12/07 [神人] 神一位正妹"
write file for "12/07 [正妹] 笑靨如花"
write file for "12/07 [正妹] 吊帶褲"
write file for "12/07 [正妹] 日本妹子"
write file for "12/07 [正妹] 忍野さら"
write file for "12/07 [神人] 神長榮正妹空姐"
write file for "12/07 [挑戰] 就一張"
write file for "12/07 [正妹] 中正高中"
write file for "12/07 [正妹] 一早來點精神……"
write file for "12/07 [神人] 神一位正妹"
write file for "12/07 [神人] 酷鎷設計model "
write file for "12/07 [正妹] 很愛笑的女孩"
write file for "12/07 [正妹]東找找西找找..........."
write file for "12/06 [正妹] 內衣模特兒"
write file for "12/06 [正妹] 志效好美啊"
write file for "12/06 [正妹

write file for "11/28 [正妹] 短髮可愛小圓臉"
write file for "11/28 [正妹] 甜甜氣質"
write file for "11/28 [正妹] 毛衣山丘"
write file for "11/28 [正妹] 黑長直"
write file for "11/28 [正妹] 睡不著"
write file for "11/28 [神人] 韓文課的韓國籍助教"
write file for "11/28 [神人] 星期日高雄FFK扮演冰雪奇緣安娜的COSER"
write file for "11/27 [神人] 影片中的女生"
write file for "11/27 [正妹] 姊"
write file for "11/27 [正妹] 小學生"
write file for "11/27 [正妹] 一個很可愛的女孩"
write file for "11/27 [正妹] 康橋國際學校"
write file for "11/27 [神人] 民視新聞記者"
write file for "11/27 [正妹] Lauren Hanley 美國模特兒 "
write file for "11/27 [正妹] 泳裝IN水池"
write file for "11/27 [正妹] 黑長髮的女孩"
write file for "11/27 [正妹] 大島優子"
write file for "11/27 [正妹] 金子理江"
write file for "11/27 [神人]第六屆 台灣成人博覽會 某SG"
write file for "11/27 [正妹] 豐唇妹子"
write file for "11/27 [神人]  養樂多VS羅德 巨乳球僮"
write file for "11/27 [正妹] 東吳美女"
write file for "11/27 [正妹] 秘書"
write file for "11/27 Re: [正妹] 氣質東京姊姊"
write file for "11/27 [神人] Samsung廣告的妹子"
write file for "11/27 [神人] 這位韓國妹是誰啊?"
write file for "11/27 [正妹] 米國念書的"
write file for "11/27 

write file for "11/15 [正妹] 冷豔但有料的竹竿"
write file for "11/15 [正妹] ㄎㄧㄤ一發"
write file for "11/15 [神人] 外國模特兒"
write file for "11/15 [正妹] 咕硓石圍牆"
write file for "11/15 [正妹] HKT48 指原莉乃"
write file for "11/15 [正妹] 女香理髮廳"
write file for "11/15 [正妹] SKE48 松井珠理奈"
write file for "11/14 [正妹] 晚安幾張"
write file for "11/14 [神人] 可以幫忙神一下這影片的女生嗎？"
write file for "11/14 [神人] 深深"
write file for "11/14 [神人] 修復硬碟時找到的翹臀"
write file for "11/14 [正妹] 到高中協助做健檢的護理師"
write file for "11/14 [正妹] 閒來無事........."
write file for "11/14 [正妹] 電影少女"
write file for "11/14 [神人] 台中全國大飯店-全壽樓服務員"
write file for "11/14 [正妹] 可愛小隻馬"
write file for "11/14 [正妹] 簡短的幾張 清秀美女"
write file for "11/14 [神人] 廣告女主角"
write file for "11/14 [正妹] 十年小池"
write file for "11/14 [正妹] 開飯..............."
write file for "11/14 [正妹] Jessica Hart 澳洲模特兒 177cm"
write file for "11/14 [神人] 據說是直播 的正妹"
write file for "11/14 [正妹] 臉書有共同好友的正妹"
write file for "11/14 [正妹] 微甜空服"
write file for "11/14 [正妹] 對三次元重新燃起希望"
write file for "11/14 [正妹] 可愛短髮"
write file for "11/14

...

write file for "2/21 [正妹] XiDo 喜多"
write file for "2/21 [神人] 新竹台大分院門口對面美芝城"
write file for "2/21 [正妹] 超激萌 第925彈"
write file for "2/21 [正妹] 北科創意設計"
write file for "2/21 [神人] 麥當勞1+1廣告展現孝心的妹子"
write file for "2/21 [正妹] 素人嫩妹"
write file for "2/21 [正妹] 甜甜笑顏 "
write file for "2/21 [正妹] 精神飽滿"
write file for "2/21 [正妹] 岡田紗佳"
write file for "2/21 [正妹] 奶特"
write file for "2/21 [正妹] 可愛"
write file for "2/21 [神人] 嗚嗚 也太難神了"
write file for "2/21 [正妹] 還是以前的美女正"
write file for "2/21 [正妹] 側乳"
write file for "2/21 [公告] boxershorts 水桶"
write file for "2/20 [美女] 杉本友美"
write file for "2/20 [正妹] 三張 "
write file for "2/20 [正妹] 短髮"
write file for "2/20 [正妹] 小隻小隻的 第924彈"
write file for "2/20 [正妹] Carolina Jaramillo 哥倫比亞模特兒 178cm"
write file for "2/20 [正妹] 雙胞胎 第一四零彈"
write file for "2/20 [正妹] 還可以的"
write file for "2/20 [廣告] 橋本梨菜日本最黑的巨乳"
write file for "2/20 [正妹] 白皙"
write file for "2/20 [帥哥] 空少"
write file for "2/20 [神人] 猛牛隊正妹"
write file for "2/20 [挑戰] 最虎財經主播"
write file for "2/20 [正妹] 為近あんな"
write file for "2

write file for "2/11 [神人] 京都清水寺看到的可愛妹"
write file for "2/11 [正妹] 兩張"
write file for "2/11 [正妹] 可愛甜甜的"
write file for "2/11 [正妹] 還不錯"
write file for "2/11 [神人] 過目不忘"
write file for "2/11 [正妹] 周冬雨一張"
write file for "2/11 [正妹] 口罩長髮 FB女孩Round 285"
write file for "2/11 [正妹] 紀念節"
write file for "2/10 [正妹] 中村静香"
write file for "2/10 [正妹] 哥倫比亞大學學生 Mirte Maas"
write file for "2/10 [正妹] 請幫我神 雙和醫院一位住院醫師"
write file for "2/10 [正妹] 會饒舌的頂級美女"
write file for "2/10 [正妹] 小心犯罪"
write file for "2/10 [正妹] 溫泉一張"
write file for "2/10 [神人] 天燈妹"
write file for "2/10 Re: [神人] 妹子(虎嫂)"
write file for "2/10 [廣告] 童顏巨乳"
write file for "2/10 [神人] 網路上逛到的妹子"
write file for "2/10 [正妹] Lauren Mellor 南非模特兒 175cm"
write file for "2/10 [正妹] 愛貓女孩 第915彈"
write file for "2/10 [神人] 比基尼女森"
write file for "2/10 [神人] 好美的記者"
write file for "2/10 [正妹] 女大18變-某松山高中的校友吧"
write file for "2/10 [神人] 神人"
write file for "2/10 [正妹] 衣張        新垣結衣打桌球"
write file for "2/10 [神人] 永豐銀行鳳山分行五號櫃"
write file for "2/10 [公告] tiger1201、BboyIllusion、cms

write file for "1/30 [正妹] 朋友的朋友"
write file for "1/29 [正妹] 氣質好 FB女孩Round 276"
write file for "1/29 [正妹] 文藝女孩"
write file for "1/29 [神人] 香港正妹"
write file for "1/29 [正妹] 大胸子瑜"
write file for "1/29 [正妹] 不錯"
write file for "1/29 [正妹] 港女"
write file for "1/29 [正妹] 女孩與熊 第907彈"
write file for "1/29 [神人] 能請問一下這個女生是誰嗎？"
write file for "1/29 [帥哥] 費神"
write file for "1/29 [正妹] 麻豆"
write file for "1/29 [正妹] 久松郁実"
write file for "1/29 [正妹] 都丸紗也華"
write file for "1/29 [正妹] 菜鳥教師 "
write file for "1/29 [正妹] 奶特"
write file for "1/29 [正妹] 甜甜的氣質女孩"
write file for "1/29 [正妹] 大眼女孩兒 第一三三彈"
write file for "1/29 [神人] UP直播黑洋裝"
write file for "1/29 [正妹] 齋藤飛鳥"
write file for "1/29 [神人] 電玩展 RO SG"
write file for "1/29 [神人] 台灣靈異事件的小鳳"
write file for "1/28 [正妹] 雞排妹一張"
write file for "1/28 [神人] 斯文女孩"
write file for "1/28 [神人] 電玩展萬代工作人員"
write file for "1/28 [正妹] 新年快樂"
write file for "1/28 [正妹] 護理師Cute FB女孩Round 275"
write file for "1/28 [帥哥] 三義勝興客棧的帥老闆"
write file for "1/28 [神人] tinder看到的正妹"
write file for "1/28 [正妹

write file for "1/19 [正妹] 微性感 第901彈"
write file for "1/19 [正妹] 甜甜的一張"
write file for "1/19 [正妹] 眼睛笑笑的 FB女孩Round 270"
write file for "1/19 [正妹] 焦圓圓"
write file for "1/19 [神人] 台中科技廠尾牙"
write file for "1/19 [神人] 神一網路正妹 500P"
write file for "1/19 [正妹] 出賣同學"
write file for "1/19 [正妹] 疑似日本妹"
write file for "1/19 [神人] 求神這位藝人"
write file for "1/19 [神人] 清新正妹"
write file for "1/19 [正妹] 北科畢業正妹空姐！"
write file for "1/19 [正妹] 還不錯"
write file for "1/19 [正妹] 喜歡你那明亮的雙眼"
write file for "1/19 [正妹] 性感的眼神"
write file for "1/19 [正妹]笑容滿滿"
write file for "1/19 Re: [神人] 台北觀光巴士車掌小姐"
write file for "1/19 [正妹] 這一期的桃園青年封面"
write file for "1/19 [正妹] Ella Purnell 空氣女孩"
write file for "1/19 [正妹] 一張決勝負"
write file for "1/18 [正妹] 緬甸好笑容"
write file for "1/18 [神人] 亞洲萬里通app入口畫面的正妹"
write file for "1/18 [正妹] 甜美空姐"
write file for "1/18 [神人] 美女記者"
write file for "1/18 [正妹] 興華高中 青春洋溢的學生妹"
write file for "1/18 [廣告] 2016 超人氣素人AV女優"
write file for "1/18 [正妹] IU 李知恩  2016年選輯"
write file for "1/18 [正妹] Fanny Francois 比利時模特兒 175cm"

write file for "1/08 [正妹] 未來的警察"
write file for "1/08 [正妹] 清秀護理師 "
write file for "1/08 [神人] 台客劇場裡的高中生"
write file for "1/08 [正妹] DJ 姊姊 FB女孩Round 264"
write file for "1/07 [正妹] 弘光科大"
write file for "1/07 [神人] 2017.01.07 勤美誠品綠園道 TOYOTA展會"
write file for "1/07 [正妹] 短髮"
write file for "1/07 [正妹] Kaye Toffee"
write file for "1/07 [正妹] 橋本マナミ"
write file for "1/07 [正妹] 岩崎舞"
write file for "1/07 [正妹] 醉鵝娘"
write file for "1/07 [神人] 夏于喬 姪女"
write file for "1/07 [正妹] 航空篇-小隻空姊 第894彈"
write file for "1/07 [神人] 這位女孩"
write file for "1/07 [正妹] 不錯"
write file for "1/07 [正妹] 德國女孩"
write file for "1/07 [神人] 阻擋抗爭的女警"
write file for "1/07 [正妹]杏沙子"
write file for "1/07 [正妹] 甜美好身材"
write file for "1/07 [神人] 今天騎士籃網youtube直播正妹"
write file for "1/07 [公告] furnaceh 水桶"
write file for "1/07 [神人] 神泳裝正妹"
write file for "1/07 [神人] 想神在ig滑到的空姐"
write file for "1/07 [正妹] 香草姊姊"
write file for "1/07 [神人] Line FB 很漂亮的女生"
write file for "1/07 [正妹] 肉特(27)"
write file for "1/06 [正妹] 伊藤しほ乃"
write file for "1/06 [正妹] 選美好身材 FB